In [9]:
import pandas as pd
from joblib import dump, load
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('divorce_data.csv', sep=';')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 55 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Q1       170 non-null    int64
 1   Q2       170 non-null    int64
 2   Q3       170 non-null    int64
 3   Q4       170 non-null    int64
 4   Q5       170 non-null    int64
 5   Q6       170 non-null    int64
 6   Q7       170 non-null    int64
 7   Q8       170 non-null    int64
 8   Q9       170 non-null    int64
 9   Q10      170 non-null    int64
 10  Q11      170 non-null    int64
 11  Q12      170 non-null    int64
 12  Q13      170 non-null    int64
 13  Q14      170 non-null    int64
 14  Q15      170 non-null    int64
 15  Q16      170 non-null    int64
 16  Q17      170 non-null    int64
 17  Q18      170 non-null    int64
 18  Q19      170 non-null    int64
 19  Q20      170 non-null    int64
 20  Q21      170 non-null    int64
 21  Q22      170 non-null    int64
 22  Q23      170 non-null    i

In [4]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
classifier = SVC(kernel='rbf', degree=5, probability=True)
classifier.fit(x_train, y_train)
pred = classifier.predict_proba(x_test)
pred_c = [[value[1]>=0.5] for value in pred]
cm = confusion_matrix(y_test, pred_c)
cm

C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinsta

array([[14,  0],
       [ 0, 20]], dtype=int64)

In [6]:
pred

array([[0.96078948, 0.03921052],
       [0.00678742, 0.99321258],
       [0.97529843, 0.02470157],
       [0.00713641, 0.99286359],
       [0.91373325, 0.08626675],
       [0.94294813, 0.05705187],
       [0.96942368, 0.03057632],
       [0.00486269, 0.99513731],
       [0.97669919, 0.02330081],
       [0.00642324, 0.99357676],
       [0.00882504, 0.99117496],
       [0.93200425, 0.06799575],
       [0.9713839 , 0.0286161 ],
       [0.00611683, 0.99388317],
       [0.00323805, 0.99676195],
       [0.9913648 , 0.0086352 ],
       [0.01271919, 0.98728081],
       [0.98543222, 0.01456778],
       [0.03183043, 0.96816957],
       [0.96406194, 0.03593806],
       [0.00730902, 0.99269098],
       [0.00882504, 0.99117496],
       [0.00599052, 0.99400948],
       [0.00819203, 0.99180797],
       [0.00921551, 0.99078449],
       [0.41267527, 0.58732473],
       [0.00718311, 0.99281689],
       [0.97080002, 0.02919998],
       [0.00676692, 0.99323308],
       [0.00607993, 0.99392007],
       [0.

In [22]:
x_test.iloc[0:1,:].values

array([[3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 0,
        3, 3, 2, 2, 0, 2, 2, 0, 0, 4]], dtype=int64)

In [8]:
dump(classifier, 'divorce_svc.pkl')

['divorce_svc.pkl']

In [10]:
mysvc = load('divorce_svc.pkl')

In [28]:
result = mysvc.predict_proba([[3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 1, 0,
        3, 3, 2, 2, 0, 2, 2, 0, 0, 4]])[0][1].round(3)
result

C:\Users\canda\anaconda3\envs\main\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


0.039

In [25]:
result

array([[0.961, 0.039]])